<article class="day-desc"><h2>--- Day 6: Trash Compactor ---</h2><p>After helping the Elves in the kitchen, you were taking a break and helping them re-enact a movie scene when you over-enthusiastically jumped into the garbage chute!</p>
<p>A brief fall later, you find yourself in a <span title="To your surprise, the smell isn't that bad.">garbage smasher</span>. Unfortunately, the door's been magnetically sealed.</p>
<p>As you try to find a way out, you are approached by a family of cephalopods! They're pretty sure they can get the door open, but it will take some time. While you wait, they're curious if you can help the youngest cephalopod with her <a href="/2021/day/18">math homework</a>.</p>
<p>Cephalopod math doesn't look that different from normal math. The math worksheet (your puzzle input) consists of a list of <em>problems</em>; each problem has a group of numbers that need to be either <em>added</em> (<code>+</code>) or <em>multiplied</em> (<code>*</code>) together.</p>
<p>However, the problems are arranged a little strangely; they seem to be presented next to each other in a very long horizontal list. For example:</p>
<pre><code>123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
</code></pre>
<p>Each problem's numbers are arranged vertically; at the bottom of the problem is the symbol for the operation that needs to be performed. Problems are separated by a full column of only spaces. The left/right alignment of numbers within each problem can be ignored.</p>
<p>So, this worksheet contains four problems:</p>
<ul>
<li><code>123</code> * <code>45</code> * <code>6</code> = <code><em>33210</em></code></li>
<li><code>328</code> + <code>64</code> + <code>98</code> = <code><em>490</em></code></li>
<li><code>51</code> * <code>387</code> * <code>215</code> = <code><em>4243455</em></code></li>
<li><code>64</code> + <code>23</code> + <code>314</code> = <code><em>401</em></code></li>
</ul>
<p>To check their work, cephalopod students are given the <em>grand total</em> of adding together all of the answers to the individual problems. In this worksheet, the grand total is <code>33210</code> + <code>490</code> + <code>4243455</code> + <code>401</code> = <code><em>4277556</em></code>.</p>
<p>Of course, the actual worksheet is <em>much</em> wider. You'll need to make sure to unroll it completely so that you can read the problems clearly.</p>
<p>Solve the problems on the math worksheet. <em>What is the grand total found by adding together all of the answers to the individual problems?</em></p>
</article>

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from aocd import get_data

puzzle_input = get_data(day=6, year=2025)
puzzle_input[:20]

'15 343  97 448 92  9'

In [3]:
sample_input = """123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  """

sample_input

'123 328  51 64 \n 45 64  387 23 \n  6 98  215 314\n*   +   *   +  '

In [4]:
[len(row.split()) for row in puzzle_input.splitlines()]

[1000, 1000, 1000, 1000, 1000]

In [5]:
from dataclasses import dataclass
from functools import reduce
from operator import add, mul
from typing import Callable


@dataclass(frozen=True, slots=True)
class Problem:
    numbers: tuple[int, ...]
    operation: Callable[[int, int], int]

    def solve(self) -> int:
        return reduce(self.operation, self.numbers)


print(Problem((123, 45, 6), mul).solve())
print(Problem((328, 64, 98), add).solve())

33210
490


In [6]:
from typing import Self


@dataclass(frozen=True, slots=True)
class Worksheet:
    problems: tuple[Problem, ...]

    @classmethod
    def from_string(cls, input_str: str) -> Self:
        numbers_str = input_str.splitlines()[:-1]
        operations_str = input_str.splitlines()[-1]
        numbers_rows = [[int(num) for num in row.split()] for row in numbers_str]
        numbers_cols = list(zip(*numbers_rows))
        operations = [mul if op == "*" else add for op in operations_str.split()]
        problems = tuple(
            Problem(tuple(nums), op) for nums, op in zip(numbers_cols, operations)
        )

        return cls(problems)

    def check_grand_total(self) -> int:
        return sum(problem.solve() for problem in self.problems)


assert Worksheet.from_string(sample_input).check_grand_total() == 4277556
Worksheet.from_string(sample_input).check_grand_total()

4277556

In [7]:
part1 = Worksheet.from_string(puzzle_input).check_grand_total()
part1

4693159084994

<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>The big cephalopods come back to check on how things are going. When they see that your grand total doesn't match the one expected by the worksheet, they realize they forgot to explain how to read cephalopod math.</p>
<p>Cephalopod math is written <em>right-to-left in columns</em>. Each number is given in its own column, with the most significant digit at the top and the least significant digit at the bottom. (Problems are still separated with a column consisting only of spaces, and the symbol at the bottom of the problem is still the operator to use.)</p>
<p>Here's the example worksheet again:</p>
<pre><code>123 328  51 64 
 45 64  387 23 
  6 98  215 314
*   +   *   +  
</code></pre>
<p>Reading the problems right-to-left one column at a time, the problems are now quite different:</p>
<ul>
<li>The rightmost problem is <code>4</code> + <code>431</code> + <code>623</code> = <code><em>1058</em></code></li>
<li>The second problem from the right is <code>175</code> * <code>581</code> * <code>32</code> = <code><em>3253600</em></code></li>
<li>The third problem from the right is <code>8</code> + <code>248</code> + <code>369</code> = <code><em>625</em></code></li>
<li>Finally, the leftmost problem is <code>356</code> * <code>24</code> * <code>1</code> = <code><em>8544</em></code></li>
</ul>
<p>Now, the grand total is <code>1058</code> + <code>3253600</code> + <code>625</code> + <code>8544</code> = <code><em>3263827</em></code>.</p>
<p>Solve the problems on the math worksheet again. <em>What is the grand total found by adding together all of the answers to the individual problems?</em></p>
</article>

In [8]:
from itertools import groupby
from typing import override


@dataclass(frozen=True, slots=True)
class Worksheet2(Worksheet):
    @override
    @classmethod
    def from_string(cls, input_str: str) -> Self:
        numbers_str = input_str.splitlines()[:-1]
        numbers_transposed = list(zip(*numbers_str))
        numbers_cols = ["".join(c) for c in numbers_transposed]
        groups = [
            [int(col.strip()) for col in g]
            for is_sep, g in groupby(numbers_cols, key=lambda x: x.strip() == "")
            if not is_sep
        ]

        operations_str = input_str.splitlines()[-1]
        operations = [mul if op == "*" else add for op in operations_str.split()]

        problems = tuple(
            Problem(tuple(nums), op) for nums, op in zip(groups, operations)
        )
        return cls(problems)


assert Worksheet2.from_string(sample_input).check_grand_total() == 3263827
Worksheet2.from_string(sample_input).check_grand_total()

3263827

In [9]:
part2 = Worksheet2.from_string(puzzle_input).check_grand_total()
part2

11643736116335